In [3]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csr_matrix

# Create a sparse matrix of shape (3, 4) with integer data type
mtx = csr_matrix((3, 4), dtype=np.int8)

# Convert sparse matrix to dense format and print it
print(mtx.todense())


[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [4]:
import numpy as np
from scipy.sparse import csr_matrix

user_ids = np.array([1, 2, 3, 4, 6, 6])
movie_ids = np.array([7, 8, 9, 7, 8, 10])
binary_ratings = np.array([1, 1, 1, 1, 1, 1])


# Create a sparse matrix
num_users = user_ids.max()
num_movies = movie_ids.max()
sparse_matrix = csr_matrix((binary_ratings, (movie_ids - 1, user_ids - 1)), shape=(num_movies, num_users))

# Filter out zero rows
non_zero_row_indices = sparse_matrix.getnnz(axis=1) > 0
sparse_matrix = sparse_matrix[non_zero_row_indices]

# Convert filtered sparse matrix to dense
filtered_sparse_matrix.todense()

NameError: name 'filtered_sparse_matrix' is not defined

In [77]:
import argparse
import numpy as np
from scipy.sparse import csr_matrix

# Set the random seed for reproducibility
def set_random_seed(seed):
    np.random.seed(seed)

# Load the dataset from the provided path and create a sparse matrix
def load_data_sparse(file_path):
    try:
        # Load raw data
        data = np.load(file_path)
        print(f"Data loaded successfully from {file_path}")
        print(f"Data shape: {data.shape}")

        # Extract user IDs and movie IDs
        user_ids = data[:, 0]
        movie_ids = data[:, 1]

        # Binary representation (1 for rated, 0 for not rated)
        binary_ratings = np.ones(len(user_ids), dtype=np.int8)

        # Create a sparse matrix
        num_users = user_ids.max()
        num_movies = movie_ids.max()
        sparse_matrix = csr_matrix((binary_ratings, (movie_ids - 1, user_ids - 1)), shape=(num_movies, num_users))

        return sparse_matrix
    except Exception as e:
        print(f"Error loading the data or creating sparse matrix: {e}")
        return None
    
def LSH_Imp (sparse_matrix):
    try:
        # Create Permutation Matrix of 
        
        PS = 100 #Permutation Size 
        mov_size = sparse_matrix.shape[0]
        usr_size = sparse_matrix.shape[1]

        #Create the Permutation Matrix with size 100 * Movies Number
        Perm_matrix = np.array([np.random.permutation(mov_size) + 1 for _ in range(PS)]).T
        print(f"Perm_matrix shape: {Perm_matrix.shape}")




        # Initialize the signature matrix with zeros values
        signature_matrix = np.zeros((PS, usr_size))

        # Compute MinHash signatures
        for perm_idx in range(PS):  # Iterate over permutations
            # Current permutation of movie indices
            perm_vector = Perm_matrix[:, perm_idx]  # 1-based indices

            # Convert to 0-based indices
            perm_vector = perm_vector - 1

            # Reorder the sparse matrix rows based on the permutation
            permuted_matrix = sparse_matrix[perm_vector, :]

            # Get the index of the first non zero 
            first_nonzero_indices = np.argmax(permuted_matrix != 0, axis=0)+1

            signature_matrix[perm_idx,] = first_nonzero_indices
        

            print(f"Processed permutation {perm_idx + 1}/{PS}")

        print("MinHash signatures computed.")
        return signature_matrix
    
    except Exception as e:
        print(f"Error loading the data or creating sparse matrix: {e}")
        return None


In [78]:
set_random_seed(42)

In [79]:
load_data_sparse('/Users/eelco/Desktop/Master Data Science/Semester 3/Advances in Data Mining/FinalAssignment/user_movie_rating.npy')

Data loaded successfully from /Users/eelco/Desktop/Master Data Science/Semester 3/Advances in Data Mining/FinalAssignment/user_movie_rating.npy
Data shape: (65225506, 3)


<17770x103703 sparse matrix of type '<class 'numpy.int8'>'
	with 65225506 stored elements in Compressed Sparse Row format>

In [80]:
signature_matrix = LSH_Imp(sparse_matrix)


Perm_matrix shape: (17770, 100)
Processed permutation 1/100
Processed permutation 2/100
Processed permutation 3/100
Processed permutation 4/100
Processed permutation 5/100
Processed permutation 6/100
Processed permutation 7/100
Processed permutation 8/100
Processed permutation 9/100
Processed permutation 10/100
Processed permutation 11/100
Processed permutation 12/100
Processed permutation 13/100
Processed permutation 14/100
Processed permutation 15/100
Processed permutation 16/100
Processed permutation 17/100
Processed permutation 18/100
Processed permutation 19/100
Processed permutation 20/100
Processed permutation 21/100
Processed permutation 22/100
Processed permutation 23/100
Processed permutation 24/100
Processed permutation 25/100
Processed permutation 26/100
Processed permutation 27/100
Processed permutation 28/100
Processed permutation 29/100
Processed permutation 30/100
Processed permutation 31/100
Processed permutation 32/100
Processed permutation 33/100
Processed permutatio

In [81]:
print(signature_matrix)

[[  4.   4.   1. ...  21.  55.  21.]
 [ 31.   7.  11. ...  11.  11.  11.]
 [ 52.   4.   4. ... 161.  21.   4.]
 ...
 [ 70.   2.  13. ...  13. 143.  13.]
 [ 50.  34.  23. ...  21.  63. 105.]
 [ 12.  38.  12. ...  53.  38.  12.]]


In [82]:
def LSH_Sim(signature_matrix, b = 10):
    #Splitting the signature matrix into (b) bands of (r) rows
    r = int(len(signature_matrix) / b)
    bands = []
    for i in range(0, len(signature_matrix), r):
        bands.append(signature_matrix[i:i+r, :])
    return bands
    

In [83]:
bands = LSH_Sim(signature_matrix)

In [84]:
print(bands)

[array([[  4.,   4.,   1., ...,  21.,  55.,  21.],
       [ 31.,   7.,  11., ...,  11.,  11.,  11.],
       [ 52.,   4.,   4., ..., 161.,  21.,   4.],
       ...,
       [ 24.,  34.,   9., ...,  63.,  17.,  30.],
       [  8.,   8.,   2., ...,  48.,  16.,   8.],
       [ 60.,  43.,  40., ...,  20.,  53.,  20.]]), array([[ 35.,  22.,  35., ...,  54.,  31.,  54.],
       [ 21.,  21.,   3., ...,  82.,  82., 127.],
       [ 24.,  29.,  29., ..., 144.,  29.,  87.],
       ...,
       [ 30.,  14.,   7., ...,  30.,  64.,  30.],
       [ 91.,   8.,  43., ...,  43.,  94.,  94.],
       [ 36.,  42.,   7., ...,  20.,  10., 122.]]), array([[ 17.,  17.,  17., ...,   2.,   9.,  17.],
       [  1.,   1.,   1., ...,   1.,  28.,   1.],
       [ 45.,  37.,  24., ...,  15.,  15.,  24.],
       ...,
       [ 63.,   5.,  12., ...,   5.,  15.,   5.],
       [ 26.,   8.,  30., ..., 204.,   5.,  85.],
       [ 35.,  77.,  38., ..., 159., 159.,  86.]]), array([[ 15.,  15.,  15., ...,  15., 183.,  15.],
       